In [1]:
import h5py
import numpy as np
import math
import keras 
import tensorflow as tf
np.random.seed(617)

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.optimizers import SGD, Adam
from keras.layers.normalization import BatchNormalization


from time import time

import pandas as pd

/home/danielchan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.8.0
2.2.0


In [3]:
import os, sys
import shutil

current_dir = os.getcwd()
HOME_DIR = current_dir


#train_data_dir = '/home/danielchan/Dropbox/ISIC2017/ISIC-2017_Test_Data'
#train_data_dir = '/home/danielchan/Downloads/train_1'
#train_data_dir = '/home/danielchan/Dropbox/ISIC2017/ISIC-2017_Validation_Data'

#prefix = 'Orig_test_features_ISIC_'
#prefix = 'Arg2_train_features_ISIC_'
#prefix = 'Orig_valid_features_ISIC_'
#prefix = 'Arg3_valid_features_ISIC_'
#prefix = 'Arg2_valid_features_ISIC_'

#train_data_dir='/home/danielchan/Downloads/train'


#train_data_dir= '/home/danielchan/Dropbox/ISIC2017/ISIC-2017_Training_Data'
#train_data_dir = '/home/danielchan/Downloads/ISIC-2017_Training_Data'

#prefix = 'Arg4_train_features_ISIC_'
#prefix = 'Orig_train_features_ISIC_'
#train_data_dir = '/home/danielchan/Downloads/Test_square'
#prefix = 'test_v2_sq_features_ISIC_'

train_data_dir = '/home/danielchan/Downloads/Train_Sq_3000'
prefix = 'Arg3000_train_features_ISIC_'

train_data_dir = '/home/danielchan/Downloads/Validation_Orig_Sq'
prefix = 'ArgSq_valid_features_ISIC_'

train_data_dir = '/home/danielchan/Downloads/Test_Data_Sq_orig'
prefix = 'Sq_orig_test_features_ISIC_'

train_data_dir = '/home/danielchan/Downloads/Validation_Data_Sq_orig'
prefix = 'Sq_orig_valid_features_ISIC_'


train_data_dir = '/home/danielchan/Downloads/Training_Data_Sq_orig'
prefix = 'Sq_orig_train_features_ISIC_'

train_data_dir = '/home/danielchan/Downloads/Training_Data_Sq_orig_added_images'
prefix = 'Sq_orig_added_images_train_features_ISIC_'

print (HOME_DIR)
print (train_data_dir)



/home/danielchan/Dropbox/ISIC2017
/home/danielchan/Downloads/Training_Data_Sq_orig_added_images


In [4]:
def vgg16_model():
    from keras.applications.vgg16 import VGG16
    from keras.applications.vgg16 import preprocess_input
    img_width, img_height = 224,224
    print ('using the VGG16 model')
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=img_input)
    print('VGG16 model loaded')
    return base_model, img_width, img_height, preprocess_input

In [5]:
def resnet50_model():
    from keras.applications.resnet50 import ResNet50
    from keras.applications.resnet50 import preprocess_input
    img_width, img_height = 224,224
    print ('using the ResNet50 model')
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=img_input)
    print('ResNet50 model loaded')
    return base_model, img_width, img_height, preprocess_input

In [6]:
def incp_v3_model(): 
    from keras.applications.inception_v3 import InceptionV3
    from keras.applications.inception_v3 import preprocess_input,decode_predictions
    img_width, img_height = 299,299
    print ('using the InceptionV3 model')
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor=img_input)
    print('InceptionV3 model loaded')
    return base_model, img_width, img_height, preprocess_input    

In [7]:
def xcp_model(): 
    from keras.applications.xception import Xception
    from keras.applications.xception import preprocess_input,decode_predictions
    img_width, img_height = 299,299
    print ('using the Xception model')
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)
    base_model = Xception(include_top=False, weights='imagenet', input_tensor=img_input)
    print('Xception model loaded')
    return base_model, img_width, img_height, preprocess_input 

In [8]:
def InceptionResNet_model():     
    from keras.applications.inception_resnet_v2 import InceptionResNetV2
    from keras.applications.inception_resnet_v2 import preprocess_input,decode_predictions
    img_width, img_height = 299,299
    print ('using the InceptionResNetV2 model')
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=img_input)
    print('InceptionResNetV2 model loaded')
    return base_model, img_width, img_height, preprocess_input 

In [9]:
def vgg19_model():
    from keras.applications.vgg19 import VGG19
    from keras.applications.vgg19 import preprocess_input
    img_width, img_height = 224,224
    print ('using the VGG19 model')
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)
    base_model = VGG19(include_top=False, weights='imagenet', input_tensor=img_input)
    print('VGG19 model loaded')
    return base_model, img_width, img_height, preprocess_input

In [ ]:
models = { 'VGG16': vgg16_model,
           'RES50': resnet50_model,
           'INCPV3': incp_v3_model,
           'VGG19':  vgg19_model,
           'IRV2': InceptionResNet_model,
            'XCP':  xcp_model}

In [ ]:

for m in models:
    train_output = prefix + str(m)
    print ('writing the data to ',train_output)
    model, img_width, img_height, preprocess_input = models[m]( )
    print ('define preprocessing function')
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  
    print (train_data_dir)
    batch   = 64
    start = time()
    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch,
            class_mode='categorical',
            shuffle=False)
    s = len(generator.filenames)
    size = int(math.ceil(s/ batch)) 
    print ('s,bacth,size= ',s,batch,size)
    print ('number of images: ',len(generator.classes))
    print ('number of images for each class: ')
    print (np.unique(generator.classes,return_counts=True))
    print (generator.class_indices)
    
    %cd '/home/danielchan/Dropbox/ISIC2017'
    print (os.getcwd())

    print ('start generating training data')            
    start = time()

    bottleneck_features_train = model.predict_generator(generator,steps=size, verbose=1, 
                                                        workers=6, use_multiprocessing=True)

    print ('time to determine train features: ',time() - start)
    output = train_output

    np.savez(open(output, 'wb'), features=bottleneck_features_train, labels=generator.classes)
    print ('training file written in ',train_output,' shape= ',bottleneck_features_train.shape)

    
    
    


writing the data to  Sq_orig_added_images_train_features_ISIC_VGG16
using the VGG16 model
VGG16 model loaded
define preprocessing function
/home/danielchan/Downloads/Training_Data_Sq_orig_added_images
Found 5000 images belonging to 3 classes.
s,bacth,size=  5000 64 79
number of images:  5000
number of images for each class: 
(array([0, 1, 2], dtype=int32), array([1374, 2372, 1254]))
{'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}
/home/danielchan/Dropbox/ISIC2017
/home/danielchan/Dropbox/ISIC2017
start generating training data
79/79 [==============================] - 17s 211ms/step
time to determine train features:  17.100415229797363
training file written in  Sq_orig_added_images_train_features_ISIC_VGG16  shape=  (5000, 7, 7, 512)
writing the data to  Sq_orig_added_images_train_features_ISIC_RES50
using the ResNet50 model
ResNet50 model loaded
define preprocessing function
/home/danielchan/Downloads/Training_Data_Sq_orig_added_images
Found 5000 images belonging to 3 classes.
s